In [ ]:
# Accessibility_Checker.ipynb
# Main entry point for the accessibility checking system

import sys
import os
from pathlib import Path
from IPython.display import display, HTML, Image, Markdown
import json
import time
from datetime import datetime

# Import URL from config
%run config.ipynb  # Import the config file

# Add the project directory to the path so we can import our modules
project_dir = Path.cwd()
if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))

# Set path for the accessibility reports
REPORTS_DIR = Path("accessibility_reports")
REPORTS_DIR.mkdir(exist_ok=True)

# Import our custom modules
module_files = [
    "Accessibility_Checker.py",
    "Data_Loader.py",
    "Enhanced_Report_Generator.py",
    "Non_Text_Content_Checker.py"  # Add the new module
]

missing_modules = []
for module_file in module_files:
    module_path = Path(module_file)
    if not module_path.exists():
        missing_modules.append(module_file)

if missing_modules:
    display(HTML(f"""
    <div style="background-color: #ffebee; padding: 10px; border-left: 4px solid #f44336; margin-bottom: 20px;">
        <h3>⚠️ Missing Required Modules</h3>
        <p>The following required module files were not found:</p>
        <ul>
            {"".join([f"<li>{module}</li>" for module in missing_modules])}
        </ul>
        <p>Please ensure that all required modules are in the same directory as this notebook.</p>
    </div>
    """))
else:
    # Now import the modules
    try:
        from Accessibility_Checker import AccessibilityTerminology, normalize_url, terminology
        from Data_Loader import load_data, count_issues_by_category
        from Enhanced_Report_Generator import (
            generate_accessibility_report, 
            create_visualization,
            get_test_instructions,
            get_fix_example
        )
        from Non_Text_Content_Checker import identify_non_text_content  # Import the new function
        
        display(HTML("""
        <div style="background-color: #e8f5e9; padding: 10px; border-left: 4px solid #4caf50; margin-bottom: 20px;">
            <h3>✅ All Modules Loaded Successfully</h3>
            <p>The accessibility checker modules have been loaded and are ready to use.</p>
        </div>
        """))
    except ImportError as e:
        display(HTML(f"""
        <div style="background-color: #ffebee; padding: 10px; border-left: 4px solid #f44336; margin-bottom: 20px;">
            <h3>⚠️ Error Loading Modules</h3>
            <p>An error occurred while importing the required modules:</p>
            <p><code>{str(e)}</code></p>
            <p>Please ensure that all required modules are in the same directory as this notebook.</p>
        </div>
        """))

# Display welcome message and instructions
display(HTML("""
<div style="background-color: #f8f9fa; padding: 20px; border-radius: 5px; margin-bottom: 20px;">
    <h1 style="color: #0275d8;">📋 Enhanced Accessibility Checker</h1>
    <h3>A comprehensive tool for evaluating web accessibility</h3>
    
    <p>This notebook runs a series of accessibility checks on a given URL and generates a detailed report.</p>
    
    <h4>Instructions:</h4>
    <ol>
        <li>Enter the URL of the website you want to check in the <code>url_to_check</code> variable in the <code>config.ipynb</code> file.</li>
        <li>Run all cells in this notebook.</li>
        <li>The accessibility report will be generated and saved in the <code>accessibility_reports</code> directory.</li>
    </ol>
</div>
"""))

# Import Selenium and related modules
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def setup_webdriver(browser='chrome'):
    """Set up the WebDriver based on the selected browser."""
    if browser.lower() == 'chrome':
        driver = webdriver.Chrome()
    elif browser.lower() == 'firefox':
        driver = webdriver.Firefox()
    elif browser.lower() == 'edge':
        driver = webdriver.Edge()
    else:
        raise ValueError(f"Unsupported browser: {browser}")
    return driver

def check_tab_order(driver, url):
    """Check the tab order of the webpage."""
    driver.get(url)
    
    # Find all focusable elements
    focusable_elements = driver.find_elements(By.CSS_SELECTOR, "a[href], button, input, select, textarea, [tabindex]")
    
    # Store the tab order issues
    tab_order_issues = []
    
    # Simulate pressing the Tab key and check the focus order
    for i in range(len(focusable_elements)):
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable(focusable_elements[i]))
            focusable_elements[i].send_keys(Keys.TAB)
            next_focused_element = driver.switch_to.active_element
            if next_focused_element != focusable_elements[(i + 1) % len(focusable_elements)]:
                tab_order_issues.append(f"Unexpected focus order: {focusable_elements[i].tag_name} -> {next_focused_element.tag_name}")
        except TimeoutException:
            tab_order_issues.append(f"Element not focusable: {focusable_elements[i].tag_name}")
    
    return tab_order_issues

def check_missing_focusable(driver, url):
    """Check for missing focusable elements."""
    driver.get(url)
    
    # Find all interactive elements
    interactive_elements = driver.find_elements(By.XPATH, "//a | //button | //input | //select | //textarea")
    
    # Store the missing focusable issues
    missing_focusable_issues = []
    
    # Check if each interactive element is focusable
    for element in interactive_elements:
        if not element.is_enabled() or not element.is_displayed():
            missing_focusable_issues.append(f"Element not focusable: {element.tag_name}")
    
    return missing_focusable_issues

def check_aria(driver, url):
    """Check for ARIA-related issues."""
    driver.get(url)
    
    # Find elements with ARIA attributes
    aria_elements = driver.find_elements(By.XPATH, "//*[@aria-label or @aria-describedby or @aria-labelledby or @role]")
    
    # Store the ARIA issues
    aria_issues = []
    
    # Check each element with ARIA attributes
    for element in aria_elements:
        if not element.is_displayed():
            aria_issues.append(f"ARIA element not visible: {element.tag_name}")
        if not element.is_enabled():
            aria_issues.append(f"ARIA element not enabled: {element.tag_name}")
        if element.get_attribute("tabindex") == "-1":
            aria_issues.append(f"ARIA element not focusable: {element.tag_name}")
    
    return aria_issues

def check_keyboard_accessibility(driver, url):
    """Check for keyboard accessibility issues."""
    driver.get(url)
    
    # Find all focusable elements
    focusable_elements = driver.find_elements(By.CSS_SELECTOR, "a[href], button, input, select, textarea, [tabindex]")
    
    # Store the keyboard accessibility issues
    keyboard_issues = []
    
    # Check each focusable element
    for element in focusable_elements:
        try:
            element.send_keys(Keys.ENTER)
            if "active" not in element.get_attribute("class"):
                keyboard_issues.append(f"Element not activated by Enter key: {element.tag_name}")
            element.send_keys(Keys.SPACE)
            if "active" not in element.get_attribute("class"):
                keyboard_issues.append(f"Element not activated by Space key: {element.tag_name}")
        except:
            keyboard_issues.append(f"Error interacting with element: {element.tag_name}")
    
    return keyboard_issues

def run_accessibility_checks(url):
    driver = setup_webdriver(browser_choice)  # Use the selected browser from config
    
    print("Running accessibility checks...")
    
    # Check tab order
    tab_order_issues = check_tab_order(driver, url)
    print(f"Tab order issues: {len(tab_order_issues)}")
    
    # Check missing focusable elements
    missing_focusable_issues = check_missing_focusable(driver, url)
    print(f"Missing focusable elements: {len(missing_focusable_issues)}")
    
    # Check ARIA-related issues
    aria_issues = check_aria(driver, url)
    print(f"ARIA-related issues: {len(aria_issues)}")
    
    # Check keyboard accessibility
    keyboard_issues = check_keyboard_accessibility(driver, url)
    print(f"Keyboard accessibility issues: {len(keyboard_issues)}")
    
    # Check for non-text content elements
    page_source = driver.page_source
    non_text_elements = identify_non_text_content(page_source)
    print(f"Found {len(non_text_elements)} non-text content elements.")
    
    driver.quit()
    
    # Generate the accessibility report
    report_data = {
        "url": url,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "tab_order_issues": tab_order_issues,
        "missing_focusable_issues": missing_focusable_issues,
        "aria_issues": aria_issues,
        "keyboard_issues": keyboard_issues,
        "non_text_content_elements": non_text_elements  # Add non-text content elements to the report
    }
    
    report_path = generate_accessibility_report(report_data)
    
    # Create visualization
    visualization_path = create_visualization(report_data)
    
    # Display the report and visualization
    display(Markdown(f"## Accessibility Report\n[Open Report]({report_path})"))
    display(Image(visualization_path))
    
    # Provide recommendations and examples
    for issue_category in ["tab_order_issues", "missing_focusable_issues", "aria_issues", "keyboard_issues"]:
        if report_data[issue_category]:
            issue_description = report_data[issue_category][0]  # Consider the first issue as an example
            test_instructions = get_test_instructions(issue_category, issue_description)
            fix_example = get_fix_example(issue_category, issue_description)
            
            display(HTML(f"""
            <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px;">
                <h4>{issue_category.replace("_", " ").title()}</h4>
                <p><strong>Example Issue:</strong> {issue_description}</p>
                <p><strong>Testing Instructions:</strong> {test_instructions}</p>
                <p><strong>Fix Example:</strong></p>
                <pre><code>{fix_example}</code></pre>
            </div>
            """))

# Run the accessibility checks
run_accessibility_checks(url_to_check)